Assignment 1:

𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒=873 + 0.0012 * 𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒 + 0.00002 * 𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒^2 − 223.57 * ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠

With a constant of 873, we would expect an observation of 0 for 'annual income' and 'have kids' to have a value of 873 for expenditure. This is likely due to the fact that an individual still has to spend money on themselves for healthcare. The -223.57 coefficient likely means that those with children spend $223.57 less on average than those without, which seems strange, although could be explained by the treatment of 'kids' as categorical, rather than continuous, since the number of children per family is probably highly skewed towards 1. 

In [30]:
# Assignment 2:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, warnings, statsmodels.formula.api as smf, statsmodels.api as sm
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, scale
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
from scipy.stats.mstats import winsorize

# Visual Preference
%matplotlib inline
warnings.filterwarnings(action='ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weather_df = pd.read_sql_query('select * from weatherinszeged', con=engine)
engine.dispose()
weather_df.head()
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
date                   96453 non-null datetime64[ns, UTC]
summary                96453 non-null object
preciptype             96453 non-null object
temperature            96453 non-null float64
apparenttemperature    96453 non-null float64
humidity               96453 non-null float64
windspeed              96453 non-null float64
windbearing            96453 non-null float64
visibility             96453 non-null float64
loudcover              96453 non-null float64
pressure               96453 non-null float64
dailysummary           96453 non-null object
dtypes: datetime64[ns, UTC](1), float64(8), object(3)
memory usage: 8.8+ MB


In [31]:
# Limit the outliers and drop non-numerical
weather_df = weather_df.select_dtypes(exclude=['datetime64[ns, UTC]'])
weather_df = weather_df.select_dtypes(exclude=['object'])

weather_df['temp_diff'] = weather_df['apparenttemperature'] - weather_df['temperature']

weather_df['w_humidity'] = winsorize(weather_df['humidity'], (0,0.10))
weather_df['w_windspeed'] = winsorize(weather_df['windspeed'], (0,0.10))
weather_df['w_pressure'] = winsorize(weather_df['pressure'], (0,0.10))
weather_df['w_apparenttemperature'] = winsorize(weather_df['apparenttemperature'], (0,0.10))
weather_df['w_temperature'] = winsorize(weather_df['temperature'], (0,0.10))
weather_df['w_temp_diff'] = winsorize(weather_df['temp_diff'], (0,0.10))
weather_df = weather_df.dropna()

In [32]:
X_weather = scale(weather_df[['w_humidity', 'w_windspeed', 'w_pressure', 'w_apparenttemperature', 'w_temperature']])
Y = weather_df['w_temp_diff']
X = sm.add_constant(X_weather)
results = sm.OLS(Y,X).fit()
results.summary()

# Bigger coefficient, greater the correlation between feature and target
# Smaller P value, difference is more likely significant

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            w_temp_diff   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 1.940e+06
Date:                Mon, 09 Sep 2019   Prob (F-statistic):               0.00
Time:                        21:04:40   Log-Likelihood:                 37631.
No. Observations:               96453   AIC:                        -7.525e+04
Df Residuals:                   96447   BIC:                        -7.519e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.1019      0.001  -2089.032      0.000      -1.103      -1.101
x1             0.0714      0.001    103.694      0.000       0.070       0.073
x2             0.0229      0.001     34.823      0.000       0.022       0.024
x3             0.0005      0.001      1.038      0.299      -0.000       0.002
x4            10.2262      0.005   1918.140      0.000      10.216      10.237
x5            -9.0358      0.005  -1680.785      0.000      -9.046      -9.025
==============================================================================
Omnibus:                    99900.266   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          6295419.448
Skew:                          -5.266   Prob(JB):                         0.00
Kurtosis:                      41.152   Cond. No.                         22.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Based on coefficients, it appears that all of the variables except for pressure have a statistically significant effect on the dependent variable due to their low p-values, with temperature and apparent temperature having the greatest effect. This makes sense as these two variables are directly related with the dependent.

In [34]:
# Create interaction variable for humidity and windspeed
weather_df['humidity_windspeed'] = weather_df.humidity * weather_df.windspeed
weather_df['w_humidity_windspeed'] = winsorize(weather_df['humidity_windspeed'], (0,0.10))

X_weather = scale(weather_df[['w_humidity', 'w_windspeed', 'w_humidity_windspeed', 'w_pressure', 'w_apparenttemperature', 'w_temperature']])
Y = weather_df['w_temp_diff']
X = sm.add_constant(X_weather)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            w_temp_diff   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 1.632e+06
Date:                Tue, 10 Sep 2019   Prob (F-statistic):               0.00
Time:                        13:26:20   Log-Likelihood:                 38066.
No. Observations:               96453   AIC:                        -7.612e+04
Df Residuals:                   96446   BIC:                        -7.605e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.1019      0.001  -2098.471      0.000      -1.103      -1.101
x1             0.0410      0.001     33.285      0.000       0.039       0.043
x2            -0.0406      0.002    -18.090      0.000      -0.045      -0.036
x3             0.0685      0.002     29.572      0.000       0.064       0.073
x4             0.0006      0.001      1.157      0.247      -0.000       0.002
x5            10.2723      0.006   1856.997      0.000      10.261      10.283
x6            -9.0814      0.006  -1630.653      0.000      -9.092      -9.070
==============================================================================
Omnibus:                    98973.273   Durbin-Watson:                   0.285
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          6069152.050
Skew:                          -5.193   Prob(JB):                         0.00
Kurtosis:                      40.447   Cond. No.                         24.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
# Assignment 3

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine2 = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

house_df = pd.read_sql_query('select * from houseprices', con=engine2)
engine.dispose()
house_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [37]:
# Create the model
house_df = house_df.select_dtypes(exclude=['object'])
house_df = house_df.dropna()
sklearn_pca = PCA(n_components=25)
Y = house_df['saleprice']
X = sklearn_pca.fit_transform(scale(house_df))
lrm = linear_model.LinearRegression()
lrm.fit(X,Y)

print('Coefficients: ', lrm.coef_)
print('Intercept: ', lrm.intercept_)

Coefficients:  [25593.8141455  -1196.9079271   2657.67417269 -4003.28016981
 -9079.14933644 -2612.12343865   811.61228236   860.23010585
 -6817.07363569 -2741.54006652 -5971.20359496 -2080.32406603
  3587.22481601  4072.45343244 -2475.35051827 -1983.86495445
 -3410.26445258  4783.33790192  4331.19118216 -1781.43029339
 -3014.99710686  2486.52748548  2561.79966398  4206.90140823
 -5017.55581772]
Intercept:  185506.15254237287


In [38]:
# Summarize
X = sm.add_constant(X)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     234.5
Date:                Tue, 10 Sep 2019   Prob (F-statistic):               0.00
Time:                        18:49:30   Log-Likelihood:                -13251.
No. Observations:                1121   AIC:                         2.655e+04
Df Residuals:                    1095   BIC:                         2.668e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.855e+05    994.615    186.511      0.000    1.84e+05    1.87e+05
x1          2.559e+04    346.908     73.777      0.000    2.49e+04    2.63e+04
x2         -1196.9079    559.584     -2.139      0.033   -2294.886     -98.930
x3          2657.6742    604.609      4.396      0.000    1471.351    3843.997
x4         -4003.2802    695.869     -5.753      0.000   -5368.668   -2637.893
x5         -9079.1493    802.669    -11.311      0.000   -1.07e+04   -7504.205
x6         -2612.1234    886.637     -2.946      0.003   -4351.823    -872.424
x7           811.6123    899.348      0.902      0.367    -953.027    2576.252
x8           860.2301    931.119      0.924      0.356    -966.749    2687.209
x9         -6817.0736    933.426     -7.303      0.000   -8648.579   -4985.568
x10        -2741.5401    945.750     -2.899      0.004   -4597.227    -885.853
x11        -5971.2036    957.317     -6.237      0.000   -7849.587   -4092.820
x12        -2080.3241    977.120     -2.129      0.033   -3997.563    -163.085
x13         3587.2248    981.778      3.654      0.000    1660.845    5513.604
x14         4072.4534   1034.373      3.937      0.000    2042.876    6102.031
x15        -2475.3505   1039.525     -2.381      0.017   -4515.037    -435.664
x16        -1983.8650   1053.109     -1.884      0.060   -4050.205      82.475
x17        -3410.2645   1083.276     -3.148      0.002   -5535.797   -1284.732
x18         4783.3379   1118.224      4.278      0.000    2589.234    6977.442
x19         4331.1912   1138.148      3.805      0.000    2097.994    6564.389
x20        -1781.4303   1151.084     -1.548      0.122   -4040.010     477.149
x21        -3014.9971   1201.378     -2.510      0.012   -5372.260    -657.734
x22         2486.5275   1238.052      2.008      0.045      57.304    4915.751
x23         2561.7997   1281.248      1.999      0.046      47.821    5075.779
x24         4206.9014   1367.195      3.077      0.002    1524.282    6889.520
x25        -5017.5558   1448.795     -3.463      0.001   -7860.285   -2174.827
==============================================================================
Omnibus:                      474.584   Durbin-Watson:                   1.908
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            72296.867
Skew:                          -0.853   Prob(JB):                         0.00
Kurtosis:                      42.306   Cond. No.                         4.18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
# How do you determine significant features from components? Do we just say component 17, 18, etc?